## Quantify regions using regionprops

In [1]:
import os
import pandas as pd
import numpy as np
from tifffile import imread
from skimage.measure import regionprops_table 

In [2]:
im_dir = r''
mask_dir = r''

output_path = r''

im_properties = ('label', 'area', 'centroid', 'bbox','mean_intensity', 'eccentricity', 'solidity')

In [3]:
# optional - exclude wells with no cells

exclude_wells = [] # example ['B8','F9','G2']
well_list = [x.split('_')[0] for x in os.listdir(mask_dir) if not(x.split('_')[0] in exclude_wells)]
well_list

['A1',
 'A4',
 'A5',
 'B11',
 'B7',
 'C10',
 'D2',
 'D3',
 'D9',
 'E4',
 'E5',
 'F11',
 'G10',
 'H1',
 'H3',
 'H7',
 'H8']

In [4]:
signal_list = list(set([x.split('_')[-1][:-4] for x in os.listdir(im_dir)]))
signal_list

['DAPI2', 'pH3', 'pCDK1', 'DAPI1', 'RB', 'Cdc25c', 'p21', 'Fib']

In [5]:
def obj_median(mask,image):
    return np.median(image[mask])

In [6]:
df_list = []
for well in well_list:

    # get the mask
    mask_path = os.path.join(mask_dir, f'{well}_mask.tif')
    mask = imread(mask_path)

    im_list = []
    for signal in signal_list:
        print(well, signal)
        im_name = [x for x in os.listdir(im_dir) if (x.startswith(well)==True and (signal in x))][0]
        im_path = os.path.join(im_dir, im_name)
        im = imread(im_path)
        im_list.append(im)

    im = np.array(im_list)
    im = np.moveaxis(im,0,2)
        
    props = regionprops_table(mask, im, properties=im_properties,extra_properties=(obj_median,))
    df = pd.DataFrame(props)

    df['well'] = well

    # calculate background
    med = np.median(im,axis=(0,1))
    for i,signal in enumerate(signal_list):
        df[f'bck_{signal}'] = med[i]

    df_list.append(df)

df = pd.concat(df_list)

A1 DAPI2
A1 pH3
A1 pCDK1
A1 DAPI1
A1 RB
A1 Cdc25c
A1 p21
A1 Fib
A4 DAPI2
A4 pH3
A4 pCDK1
A4 DAPI1
A4 RB
A4 Cdc25c
A4 p21
A4 Fib
A5 DAPI2
A5 pH3
A5 pCDK1
A5 DAPI1
A5 RB
A5 Cdc25c
A5 p21
A5 Fib
B11 DAPI2
B11 pH3
B11 pCDK1
B11 DAPI1
B11 RB
B11 Cdc25c
B11 p21
B11 Fib
B7 DAPI2
B7 pH3
B7 pCDK1
B7 DAPI1
B7 RB
B7 Cdc25c
B7 p21
B7 Fib
C10 DAPI2
C10 pH3
C10 pCDK1
C10 DAPI1
C10 RB
C10 Cdc25c
C10 p21
C10 Fib
D2 DAPI2
D2 pH3
D2 pCDK1
D2 DAPI1
D2 RB
D2 Cdc25c
D2 p21
D2 Fib
D3 DAPI2
D3 pH3
D3 pCDK1
D3 DAPI1
D3 RB
D3 Cdc25c
D3 p21
D3 Fib
D9 DAPI2
D9 pH3
D9 pCDK1
D9 DAPI1
D9 RB
D9 Cdc25c
D9 p21
D9 Fib
E4 DAPI2
E4 pH3
E4 pCDK1
E4 DAPI1
E4 RB
E4 Cdc25c
E4 p21
E4 Fib
E5 DAPI2
E5 pH3
E5 pCDK1
E5 DAPI1
E5 RB
E5 Cdc25c
E5 p21
E5 Fib
F11 DAPI2
F11 pH3
F11 pCDK1
F11 DAPI1
F11 RB
F11 Cdc25c
F11 p21
F11 Fib
G10 DAPI2
G10 pH3
G10 pCDK1
G10 DAPI1
G10 RB
G10 Cdc25c
G10 p21
G10 Fib
H1 DAPI2
H1 pH3
H1 pCDK1
H1 DAPI1
H1 RB
H1 Cdc25c
H1 p21
H1 Fib
H3 DAPI2
H3 pH3
H3 pCDK1
H3 DAPI1
H3 RB
H3 Cdc25c
H3 p21
H3 Fib
H7 DAPI2

### Raname mean and median columns to include signal names

In [9]:
ind = 0
ind_med = 0
new_cols = []
for col in df.columns:
    if 'mean_intensity' in col:
        col = 'mean_'+signal_list[ind]
        ind += 1
        new_cols.append(col)
    elif 'obj_median' in col:
        col = 'med_'+signal_list[ind_med]
        ind_med += 1
        new_cols.append(col)
    else:
        new_cols.append(col)

In [10]:
new_cols

['label',
 'area',
 'centroid-0',
 'centroid-1',
 'bbox-0',
 'bbox-1',
 'bbox-2',
 'bbox-3',
 'mean_DAPI2',
 'mean_pH3',
 'mean_pCDK1',
 'mean_DAPI1',
 'mean_RB',
 'mean_Cdc25c',
 'mean_p21',
 'mean_Fib',
 'eccentricity',
 'solidity',
 'med_DAPI2',
 'med_pH3',
 'med_pCDK1',
 'med_DAPI1',
 'med_RB',
 'med_Cdc25c',
 'med_p21',
 'med_Fib',
 'well']

In [11]:
df.columns = new_cols

In [12]:
df.to_csv(output_path,index=False)